# BioExplorer - Atlas
![](../bioexplorer_vasculature_banner.png)

### Connect to back-end

In [ ]:
from bioexplorer import BioExplorer

url = 'localhost:5000'
be = BioExplorer(url)
core = be.core_api()
status = be.reset_scene()

### Load Atlas

In [91]:
atlas_assembly_name = 'Atlas'
print(be.remove_assembly(atlas_assembly_name))
atlas_assembly = be.add_assembly(atlas_assembly_name)
atlas_model = be.add_atlas(
    assembly_name=atlas_assembly_name,
    load_cells=True, cell_radius=5.0,
    region_sql_filter='level=9',
    cell_sql_filter='guid%10=0',
    # cell_sql_filter='x>-200 and x<200'
    # sql_filter='cell_type_guid=5'
)

In [ ]:
be.reset_camera()
core.set_renderer()

In [ ]:
status = core.set_renderer(
    current='bio_explorer',
    subsampling=4, max_accum_frames=64)
params = core.BioExplorerRendererParams()
params.gi_distance = 1000.0
params.shadows = 1.0
params.soft_shadows = 1.0
params.exposure = 2.0
status = core.set_renderer_params(params)

In [ ]:
import json
import os
data_folder = '/gpfs/bbp.cscs.ch/project/proj129/atlas/mouse/v2'
f = open(os.path.join(data_folder, 'brain_regions.json'))
obj = json.load(f)

colors=dict()

def display_child(node):
    node_id = node['id']
    color = node['color_hex_triplet']
    colors[node_id] = color
    for child in node['children']:
        display_child(child)

for node in obj['msg']:
    display_child(node)


In [ ]:
import seaborn as sns

def hex_to_rgb(value):
    value = value.lstrip('#')
    lv = len(value)
    return tuple(int(value[i:i + lv // 3], 16) for i in range(0, lv, lv // 3))

model_ids = be.get_model_ids()['ids']
model_id = model_ids[len(model_ids)-1]
material_ids = be.get_material_ids(model_id)['ids'][:-1]
nb_materials = len(material_ids)
palette = list()
for material_id in material_ids:
    c = hex_to_rgb(colors[material_id])
    palette.append([c[0] / 255.0, c[1] / 255.0, c[2] / 255.0])
be.set_materials(
    model_ids=[model_id], material_ids=material_ids,
    diffuse_colors=palette, specular_colors=palette)
core.set_renderer()

In [ ]:
# transformation = {
#     'rotation': [-0.5, 0.5, -0.5, 0.5],
#     'rotation_center': [0.0, 0.0, 0.0],
#     'scale': [1.0, 1.0, 1.0],
#     'translation': [-8500.0, 4000.0, 5000.0]
# }
# core.update_model(0, transformation=transformation)
